# Trading Day Of the Week (TDOW)

Vamos a analizar cuantitativamente la serie historica de precios de **BTC** (5 años aprox) para observar **la probabilidad de que el cierre diario del activo sea al alza o baja segun el dia de la semana.** Esto nos va a servir como un indicador para tomar en cuenta al momento de operar, y tambien va a servir para explicar que los mercados no se comportan como **random walk** debido a que estan predispuesto a cerrar arriba o abajo segun el tipo del dia. Aparte de calcular la probabilidad, vamos a calcular **el promedio del rendimiento segun cada dia de la semana.** Se tomaron datos de Abril 2013 a Abril 2018.

### Importamos los paquetes y librerias necesarios para el analisis

In [1]:
import pandas as pd
import numpy as np

### Cambio el directorio a la carpeta "Datasets" donde voy a tener el archivo de los datos historicos

In [2]:
cd C:\Users\david\Documents\Datasets

C:\Users\david\Documents\Datasets


### Corroboro que la direccion presente de trabajo sea igual al directorio cambiado, donde estan las series

In [3]:
pwd

'C:\\Users\\david\\Documents\\Datasets'

### Importo el CSV y lo agrego a un dataframe llamado "data"

In [4]:
data = pd.read_csv('BTC5Y.csv',low_memory=False)

### Uso .head() para visualizar las 10 primeras filas del dataframe

In [5]:
data.head()

,Id,CoinId,Symbol,Date,Open,High,Low,Close,Volume,MarketCap
0,835561,1,BTC,2013-04-28 00:00:00.000,135.300003,135.979996,132.100006,134.210007,0.0,1500519936
1,835560,1,BTC,2013-04-29 00:00:00.000,134.444000,147.488007,134.000000,144.539993,0.0,1491160064
2,835559,1,BTC,2013-04-30 00:00:00.000,144.000000,146.929993,134.050003,139.000000,0.0,1597779968
3,835558,1,BTC,2013-05-01 00:00:00.000,139.000000,139.889999,107.720001,116.989998,0.0,1542819968
4,835557,1,BTC,2013-05-02 00:00:00.000,116.379997,125.599998,92.281898,105.209999,0.0,1292189952


### Observo la forma de la serie (filas x columnas)

In [6]:
data.shape

(1925, 10)

### Mido la cantindad de datos/variables unicas que existen en la columna de "Symbol"

Deberia ser 1, ya que solo estamos analizando el activo de BTC. Si sale 1, comprobamos que en ese sentido, la serie esta bien.

In [7]:
len(data['Symbol'].unique())

1

### Busco si faltan valores, True=Si False=No

Como aparece todo False (No), comprobamos que en este otro sentido, la serie esta "limpia".

In [8]:
data.isnull().any()

Id           False
CoinId       False
Symbol       False
Date         False
Open         False
High         False
Low          False
Close        False
Volume       False
MarketCap    False
dtype: bool

In [9]:
data.head()

,Id,CoinId,Symbol,Date,Open,High,Low,Close,Volume,MarketCap
0,835561,1,BTC,2013-04-28 00:00:00.000,135.300003,135.979996,132.100006,134.210007,0.0,1500519936
1,835560,1,BTC,2013-04-29 00:00:00.000,134.444000,147.488007,134.000000,144.539993,0.0,1491160064
2,835559,1,BTC,2013-04-30 00:00:00.000,144.000000,146.929993,134.050003,139.000000,0.0,1597779968
3,835558,1,BTC,2013-05-01 00:00:00.000,139.000000,139.889999,107.720001,116.989998,0.0,1542819968
4,835557,1,BTC,2013-05-02 00:00:00.000,116.379997,125.599998,92.281898,105.209999,0.0,1292189952


### Cambio el formato de los datos de la columna "Date" al de fecha mediante pd.to_datetime

In [10]:
data['Date'] = pd.to_datetime(data['Date'])

### Una vez que esta en formato datetime, puedo cambiar las fechas segun dia de semana

Utilizo .head() para ver las primeras 5 filas segun el formato del dia de semana

In [11]:
data['Date'].dt.weekday.head()

0    6
1    0
2    1
3    2
4    3
Name: Date, dtype: int64

### Creo un diccionario para asignarle un string value al formato numerico del dia de semana

In [12]:
day_dict = {
    0:'Lunes',
    1:'Martes',
    2:'Miercoles',
    3:'Jueves',
    4:'Viernes',
    5:'Sabado',
    6:'Domingo'
           }

### Agrego una columna para asignarle el dia de semana a cada fecha, utilizando el diccionario que creamos antes

In [13]:
data['Day_of_week']=data['Date'].dt.weekday.map(day_dict)

In [14]:
data.head()

,Id,CoinId,Symbol,Date,Open,High,Low,Close,Volume,MarketCap,Day_of_week
0,835561,1,BTC,2013-04-28,135.300003,135.979996,132.100006,134.210007,0.0,1500519936,Domingo
1,835560,1,BTC,2013-04-29,134.444000,147.488007,134.000000,144.539993,0.0,1491160064,Lunes
2,835559,1,BTC,2013-04-30,144.000000,146.929993,134.050003,139.000000,0.0,1597779968,Martes
3,835558,1,BTC,2013-05-01,139.000000,139.889999,107.720001,116.989998,0.0,1542819968,Miercoles
4,835557,1,BTC,2013-05-02,116.379997,125.599998,92.281898,105.209999,0.0,1292189952,Jueves


### Creo una columna para calcular el PnL diario

Calculamos la ganancia que obtendriamos al comprar a la apertura y vender al cierre

In [15]:
data['PnL'] = data['Close'] - data['Open']

### Creo una columna para referirme si el PnL fue negativo o postivio
0 = Negativo
1 = Positivo

In [16]:
data['Pos_neg'] = data['PnL'].apply(lambda x: int(x >=0))

In [17]:
data.head()

,Id,CoinId,Symbol,Date,Open,High,Low,Close,Volume,MarketCap,Day_of_week,PnL,Pos_neg
0,835561,1,BTC,2013-04-28,135.300003,135.979996,132.100006,134.210007,0.0,1500519936,Domingo,-1.089996,0
1,835560,1,BTC,2013-04-29,134.444000,147.488007,134.000000,144.539993,0.0,1491160064,Lunes,10.095993,1
2,835559,1,BTC,2013-04-30,144.000000,146.929993,134.050003,139.000000,0.0,1597779968,Martes,-5.000000,0
3,835558,1,BTC,2013-05-01,139.000000,139.889999,107.720001,116.989998,0.0,1542819968,Miercoles,-22.010002,0
4,835557,1,BTC,2013-05-02,116.379997,125.599998,92.281898,105.209999,0.0,1292189952,Jueves,-11.169998,0


### Creo una columna para calcular los retornos logaritmicos del PnL (comprar a la apertura y vender al cierre)

In [18]:
data['Log_Ret_PnL'] =  np.log(data['Close']/data['Open'])
data.head()

,Id,CoinId,Symbol,Date,Open,High,Low,Close,Volume,MarketCap,Day_of_week,PnL,Pos_neg,Log_Ret_PnL
0,835561,1,BTC,2013-04-28,135.300003,135.979996,132.100006,134.210007,0.0,1500519936,Domingo,-1.089996,0,-0.008089
1,835560,1,BTC,2013-04-29,134.444000,147.488007,134.000000,144.539993,0.0,1491160064,Lunes,10.095993,1,0.072408
2,835559,1,BTC,2013-04-30,144.000000,146.929993,134.050003,139.000000,0.0,1597779968,Martes,-5.000000,0,-0.035339
3,835558,1,BTC,2013-05-01,139.000000,139.889999,107.720001,116.989998,0.0,1542819968,Miercoles,-22.010002,0,-0.172385
4,835557,1,BTC,2013-05-02,116.379997,125.599998,92.281898,105.209999,0.0,1292189952,Jueves,-11.169998,0,-0.100902


### Sumamos los Pos_neg segun el dia de semana
Esto nos va a permitir saber cuantos dias son los positivos, sobre el total de observaciones y la probabilidad que el PnL (Open-Close) sea positiva

In [19]:
group1 = data.groupby(['Day_of_week']).agg(['sum','size'])['Pos_neg']
group1['Prob_PnL +'] = group1['sum']/group1['size']
group1

,sum,size,Prob_PnL +
Day_of_week,,,
Domingo,140,275,0.509091
Jueves,148,275,0.538182
Lunes,157,275,0.570909
Martes,151,275,0.549091
Miercoles,137,275,0.498182
Sabado,157,275,0.570909
Viernes,155,275,0.563636


**Los dias mas propensos a cerrar al alza son los Lunes y Sabado con un 57.09% de probabilidad. Mientras que el unico dia historicamente predispuesto a cerrar a la baja son los miercoles con un 1-0.498182 = 0.501818% que es igual a cerrar casi neutro.**

### Agrupamos segun dia de semana y segun si el PnL fue negativo o positivo

Ademas de eso, calculamos la media de los retornos logaritmicos segun el Pos_neg

In [20]:
data.groupby(['Day_of_week','Pos_neg']).agg(['mean'])[['Log_Ret_PnL']]

Log_Ret_PnL
                           mean
Day_of_week Pos_neg            
Domingo     0         -0.023459
            1          0.023709
Jueves      0         -0.033280
            1          0.032173
Lunes       0         -0.024538
            1          0.031609
Martes      0         -0.028472
            1          0.028233
Miercoles   0         -0.031667
            1          0.027054
Sabado      0         -0.025042
            1          0.024316
Viernes     0         -0.033283
            1          0.025606

**Como se analizo previamente, los lunes y sabados que son los dias mas inclinados historicamente a cerrar al alza. Los mismos tienen un promedio de rendimiento de 3.16% y 2.43% en estos dias de PnL positivos.**